# Introduction to Word Embedding 

Natural Language Processing (NLP) is now everywhere. From Google search, Google translate, analyzing product reviews in e-commerce sites like Amazon or in doing sentiment analysis on particular topics in Twitter, using machine learning for NLP is becoming more important every minute. There's many popular ways to transform raw text to numbers to use in a machine learning model such as bag-of-words or Term Frequency Inverse Document Frequency (TF-IDF). 

The limitation with most of these methods is that they don't capture meaning. If you only count the number of times each word appears in the text (bag-of-words) or even weight it with number of times those words appear in the documents (TF-IDF), you still do not have any notion about which words are truly similar or share similar meaning. Word Embedding is an unsupervised technique for capturing word meanings, understanding semantic relationships between them by creating representation for words. 

This tutorial will cover the following topics: 

* Definition of word embeddings
* How to train our own word embeddings with `gensim`
* Visualizing the word embeddings using t-SNE
* Using pre-trained word vectors
* How to use word embeddings for text classification

## What Are Word Embeddings? (NEED TO REWRITE USING MORE DIAGRAMS)

Word embeddings (also called "word vectors") are mappings from words to real valued vectors. The key idea is that each word will be represented by a numeric vector. It is a dense representation of words in a low dimensional vector space.

If you simply want to represent a word by a vector, you can use the one-hot-representation. Here, the length of the vector corresponds to the size of total vocabulary. Each vector will have a non-zero-entry(1) in its place in the vocabulary. For example, if you have a three-word vocabulary ```{book, king, queen}```, the three word vectors will look like this:

```
book = [1,0,0]
king = [0,1,0]
queen = [0,0,1]
```

However, if you have one million words in your vocabulary, then you have vectors of 1 million length with only one non-zero-entry which would be very sparse vectors. One-hot-representations work well for discrete values of categorical data well, but they can also be inefficient when the vectors are very sparse. This takes a huge space in memory and when it comes to computing with these vectors (that is, doing operations such as addition, multiplication, ...), it'd take a lot of time.

On top of that, one-hot-representations do not preserve similarity or relationships between words. The vector for `king` and the vector for `queen` does not have any similarity between them. If you take the dot product between these two vectors, you will end up with a zero even if both words appear in similar contexts. 

<!--- One-hot-representations do not preserve semantic relationships. = sort of a repetition of the similarity/relationship between words?--->

The solution to this problem is then to create a "distributed representation" that preserves the notion of similarity in a low-dimensional space. Word embeddings create a low dimensional(length of 50,100,300 etc instead of 1M+ <!-- where to these numbers come from? --->) vector containing real numbers where the meaning of the words is distributed through all the dimensions in the vector instead of just having one non-zero entry. 

With this solution in place, your word vectors could look like this : 

```
book = [.34,.56,.22]
king = [.94,.75,.83]
queen = [.92,.70,.78]
```


<!-- I'm missing the wider context of word embeddings here. 

* Mainly the link to the history (why and how people moved to using word embeddings and what the first approaches were. : distributional semantics

* Link to the broader context of NLP: frequency based vs prediction based embeddings

https://www.gavagai.se/blog/2015/09/30/a-brief-history-of-word-embeddings/ 
term "word embedding" might not completely be applicable: http://ruder.io/word-embeddings-1/: "Bengio et al. coin the term word embeddings in 2003 and train them in a neural language model jointly with the model's parameters." 
https://arxiv.org/abs/1705.01509 

* Link from low dimensional vectors to the more advanced research that has been done and that is reflected in the next markdown cell
---> 

# Classic Word Embedding Models


The rest of this tutorial will provide you with a brief overview of the popular word embedding models and you'll learn more about some popular software for training and using word embeddings:

* ```Word2vec``` :  Word2Vec was released by a team of researchers at Google led by Tomas Mikolov in 2013. Since then many other researchers have also explained word2vec...[more here]


* ```Glove``` : [Glove](https://nlp.stanford.edu/projects/glove/) (Global Vectors for Word Representation) was created by Stanford NLP group. They provide many pre-trained vectors of different dimensions(50,100,200 or 300 dimensions) trained on Wikipedia and Twitter corpus for general use. [more here]


* ```FastText``` : [FastText](https://github.com/facebookresearch/fastText) is the latest word embedding library released by Facebook in 2016. Unlike Word2Vec and Glove, FastText treats words as composed of its character n-grams. Vectors created for one word is the sum of vectors created for its character n-grams. For example, the vector for orange might be the sum of the vectors of the character level bigrams like ```or|ra|an|ng|ge|e```  and trigrams like ```ora|ran|ang|nge|ge|e```. This helps to generate better representations for out-of-vocabulary words and rare words because even if the word does not appear in the corpus or appears very infrequently it can still have character n-grams shared with other words. FastText also released pre-trained word vectors for 294 languages trained on wikipedia corpus in 2017.

## Training Word Embeddings

Now that you have some idea about what word embeddings are, you can easily train our own word embeddings using Gensim wrapper for [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html); Gensim is a Python library for topic modelling, document indexing and similarity retrieval with large corpora. After, you'll visualize the embeddings using `t-sne` from `scikit-learn`. 

The dataset that you will use comes from [Project Gutenberg](https://www.gutenberg.org/). [Pride and Prejudice](https://en.wikipedia.org/wiki/Pride_and_Prejudice) is a famous romance novel written by [Jane Austen](https://en.wikipedia.org/wiki/Jane_Austen), a very well-known author from late 18th century. The book was first published in 1813. Later, many modern movies and dramas have been made out of the book. 

![ ](images/ja.jpg)

You'll get the data from Project Gutenberg and preprocess the data for making it ready for the `gensim` word2vec model. Then we will train the word embeddings with `gensim`'s word2vec.

First, you'll load all the libraries that you may need. Here's a brief review of how each library will be used : 

* `gensim` : To create word vectors
* `requests` : To get the data from Project Gutenberg
* `re` : Regex for cleaning the dataset
* `numpy`, `sklearn`, and `matplotlib` : For plotting the word vectors in 2 dimensions with t-SNE

In [ ]:
import gensim
import requests
import re
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

### Data Preprocessing

Here, you'll need to make some helper functions so that you can collect the text of the book from Project Gutenberg. 
* The function `get_book()` gets the pride and prejudice text from the Project Gutenberg website and discards some extra text for processing. 

* The `preprocess()` function keeps only the alpha-numeric characters and removes all special characters. It also makes the text lowercase and splits it into a list of sentences.

* The `get_book_corpus()` function uses both of the helper functions `get_book()` and `preprocess()` to get the book text as a list of sentences. Then it breaks up the sentences into words for tokenization and returns the corpus. 

In [ ]:
pride_and_prejudice_url = 'http://www.gutenberg.org/files/1342/1342-0.txt'

def get_book(url):
    # gets the text from project gutenberg with a http request
    raw = requests.get(url).text
    # each book in project gutenberg has some metadata in the beginning so we discard them
    start = re.search(r"\*\*\* START OF THIS PROJECT GUTENBERG EBOOK .* \*\*\*",raw ).end()
    # similarly there's some extra text in the end too, so we discard them
    stop = re.search(r"End of the Project Gutenberg EBook", raw).start()
    # we only keep the relevant text
    text = raw[start:stop]
    return text

def preprocess(sentence): 
    return re.sub('[^A-Za-z0-9.]+', ' ', sentence).lower().split(".")
    
def get_book_corpus(book_url):
    book = get_book(book_url)
    processed_book = preprocess(book)
    corpus = [sentence.split(" ") for sentence in processed_book]
    return corpus

In [ ]:
pride_and_prejudice = get_book_corpus(pride_and_prejudice_url)
corpus = pride_and_prejudice

As you can see, you have the word tokenized version of the famous first two opening sentences from "Pride and Prejudice" with some extra header words.

In [ ]:
print(corpus[0:2])

There's some uninformative words in text like blank space (`""`) and many stopwords like "a", "the", etc. These words occur frequently but do not add much to the meaning of the sentences. In essence, it's not a big problem when you are training on millions of news articles and want to improve classification accuracy on something like Google Search. However, in this case, you are going to visualize the embeddings, so it's better if you can get a good vocabulary set. You get the final corpus after removing the stopwords with the help of [NLTK](http://www.nltk.org/), the de-facto libary for Natural Language Processing (NLP) in Python.

In [ ]:
from nltk.corpus import stopwords
stopwords_list = stopwords.words('english') + ['']

final_corpus = []
for sentence in corpus:
    dummy = []
    for word in sentence:
        if word not in stopwords_list:
            dummy.append(word)
    final_corpus.append(dummy)

## Training Word2Vec Model

In this section, you'll briefly go over the details of the word2vec model. First you'll get familiar with some terminology like "context words","center word" and "window size". 

Let's consider an example sentence like the following: "I love ham and egg sandwich with coffee for breakfast."

For each word in the sentence, you can consider that word as the 'center' and the surrounding words as the context words. If you pick 'coffee' as the center word with a window size of 1 for the context words, then 'with' and 'for' are the context words. Similarly, if you pick 'love' as the center word, then 'I' and 'ham' will be the context words. The window size is simply the number of words to the left and right of the center word that should be considered as context words. With a window size of 2 instead of 1, the context words for "coffee" become {sandwich, with} and {for, breakfast}.

Word2Vec can be trained with two architectures, namely, the Continuous Bag of Words(CBOW) and skip-gram model. On the one hand, CBOW predicts the current or center word given the window of the context words. The order of the context words does not impact the prediction. Skip-gram, on the other hand, predicts the context words for a given window size given a center word. CBOW is faster to train but skip-gram is slightly better in predicting infrequent words.

Gensim provides many parameters to control the training of the Word2Vec model.

* `sentences` : An iterable containing the corpus for training.
* `sg`: 0 or 1. It defines the training algorithm. Default value is 0 and CBOW model is used. If you use 1 then skip-gram model is used.
* `size` : The dimension of the embeddings that you specify. 300 will give us a 300 dimensional vector, 150 will give us a 150 dimensional vector for each word and so on.
* `window` : Maximum distance between the current word and predicted word in a sentence.
* `min_count` : Words with lower frequency than the min_count will be discarded.
* `alpha` : Initial learning rate. Default is 0.025
* `seed` : Random seed for getting the same results later.

In [ ]:
model = gensim.models.Word2Vec(final_corpus, size=300, window=8, min_count=10,seed=4)

## Using Trained Embeddings

After the model has been trained, you can check the word vector for some particular word. "Darcy" is one of the main characters in the novel "Pride and Prejudice", so let's check his word vector. As expected, you have a 300 dimensional real-valued vector for the embedding.

In [ ]:
model['darcy']

You can check how similar two words are with respect to occuring in the same context with the `similarity()` method. "Darcy" and "Elizabeth" occur together frequently, which explains why their similarity is quite high. 

In [ ]:
model.wv.similarity('darcy','elizabeth')

You can see which words are most similar to Elizabeth with the help of the `most_similar()` method. First one is Lydia, her sister.

In [ ]:
model.wv.most_similar('elizabeth')

Similarly, you can check the similar words for "Darcy". The "letter" refers to parts in the novel where two of the main characters, Darcy and Elizabeth, have some heated discussions about a letter. "Mr. Collins", "Lydia" and "Mr. Bennet" refer to other characters of the novel.

In [ ]:
model.wv.most_similar('darcy')

In [ ]:
model.wv.most_similar('bingley')

You can check how many words were included in the final vocabulary.

In [ ]:
len(model.wv.vocab)

You can also check which words were included in the vocabulary with the `keys()` method.

In [ ]:
model.wv.vocab.keys()

According to the documentation, there is a `gensim.models.phrases` module which lets you automatically detect phrases longer than one word. Using the `phrases` module, you can learn where "words" are actually multi-word expressions, such as `new_york_times` or `financial_crisis`. You can give this a try for the "Pride and Prejudice" novel, too: 

In [ ]:
bigram_transformer = gensim.models.Phrases(final_corpus)
model2 = gensim.models.Word2Vec(bigram_transformer[final_corpus],size=300, window=6, min_count=90,seed = 70)

In [ ]:
model2.wv.most_similar("collins")

<!-- small discussion of the results of the phrases module ---> 
With your embeddings, you can create visualizations and even perform text classifications.


# Visualizing Word Embeddings With t-SNE 

t-Distributed Stochastic Neighbor Embedding (t-SNE) is a visualization method that is frequently used for visualizing high dimensional vectors in 2D or 3D space. 

Note that the axis in the t-SNE visualization doesn't mean anything, rather the relative positions of the vectors show which vectors are closer in the high-dimensional space. <!--- I don't understand where "axis" comes from? ---> 

You have created embeddings of 300 dimensions, but you can visualize them with t-SNE in the 2D space. You use a limit of 400 here because visualizing all the words can look a bit cluttered.

In [ ]:
limit = 400
vector_dim = 300

# Get tokens and vectors
words = []
embedding = np.array([])
i = 0
for word in model.wv.vocab.keys():
    # Break the loop if limit exceeds 
    if i == limit: break

    # Get token 
    words.append(word)

    # Append the vectors 
    embedding = np.append(embedding, model[word])

    i += 1

# Reshape the embedding vector 
embedding = embedding.reshape(limit, vector_dim)


def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')
    plt.savefig(filename)


# Create the t-SNE plot [Warning: will take time]
tsne = TSNE(perplexity=30.0, n_components=2, init='pca', n_iter=5000)

low_dim_embedding = tsne.fit_transform(embedding)

# Plot and save the figure
plot_with_labels(low_dim_embedding, words)

# Pre-trained word embeddings 


In this part, you'll see how you can download, load and use Google's pretrained word vectors in `gensim`. These word vectors are 300 dimensional and they were trained on a corpus of Google News Dataset with around 100 billion words. They have word vectors for a vocabulary of around 3 million words.

In [ ]:
import gensim

# Using pre-trained word embeddings for classification with Keras :

In this section we'll participate in 2017 Halloween themed Kaggle Competition ["Spooky Author Identification"](https://www.kaggle.com/c/spooky-author-identification) where the challenge is to predict the author of excerpts from horror stories by Edgar Allan Poe, Mary Shelley, and HP Lovecraft using word embeddings and keras.

![](images/hl.jpg)

It's a multiclass classification problem with 3 classes - EAP, HLP and MWS , short form of the author names. The metric for evaluation is multiclass logarithmic loss/categorical crossentropy, a very common metric for multiclass classification models. 

The competition dataset contains text from works of fiction written by spooky authors of the public domain: Edgar Allan Poe, HP Lovecraft and Mary Shelley. The data was prepared by chunking larger texts into sentences using CoreNLP's MaxEnt sentence tokenizer, so you may notice the odd non-sentence here and there. The objective is to accurately identify the author of the sentences in the test set. The dataset can be downloaded from [Kaggle](https://www.kaggle.com/c/spooky-author-identification/data)

